In [1]:
#%%
! pip install dgl-cu101 ogb
ROOT = "/kaggle/input/obgnproteins-dgl-data/"

     |████████████████████████████████| 25.5 MB 356 kB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
  Created wheel for outdated: filename=outdated-0.2.0-py3-none-any.whl size=4960 sha256=a84b18539a9dbbadd36ab7d162aa0c6d2098e0c9338cf1920f330c230168b058
  Stored in directory: /root/.cache/pip/wheels/6f/cd/a2/e49170b2cf59e88b952f3414f25a54d9f16f033bded4aaab26
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=6f38473ba0be5a240b5f9117d2724acaa79bbb323889b4d8ea4cf5a332c59a03
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built outdated littleutils
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from dgl.data.utils import load_graphs
import numpy as np
import random
import os
import math
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.function as fn
from dgl import DGLGraph
from dgl.nn import GraphConv, SAGEConv
from ogb.nodeproppred import Evaluator
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau

x,_ = load_graphs(ROOT + "data.bin")
graph = x[0]

train_idx = torch.load(ROOT + 'train.pt')
test_idx = torch.load(ROOT + 'test.pt')
valid_idx = torch.load(ROOT + 'valid.pt')

splitted_idx = {'train':train_idx, 'test':test_idx, 'valid':valid_idx}

device = torch.device('cuda')

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


In [3]:
# copy mean of edge features to node
# split node feature for every channel of the edge
graph.update_all(fn.copy_e('feat','m'),fn.sum('m','edata_mean'))

n_feat = graph.ndata['edata_mean'].shape[1]

for channel in range(n_feat):
        graph.ndata['feat_' + str(channel)] = graph.ndata['edata_mean'][:, channel:channel+1]


In [4]:
def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    print ('random seed set to be ' + str(seed))

def run_a_train_epoch(graph, node_idx, model, criterion, optimizer, evaluator):
    model.train()
    logits = model(graph)[node_idx]
    labels = graph.ndata['labels'][node_idx]
    loss = criterion(logits, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss = loss.data.item()
    labels = labels.cpu().numpy()
    preds = logits.cpu().detach().numpy()

    return loss, evaluator.eval({"y_true": labels, "y_pred": preds})['rocauc']

def run_an_eval_epoch(graph, splitted_idx, model, evaluator):
    model.eval()
    with torch.no_grad():
        logits = model(graph)
    labels = graph.ndata['labels'].cpu().numpy()
    preds = logits.cpu().detach().numpy()

    train_score = evaluator.eval({
        "y_true": labels[splitted_idx["train"]],
        "y_pred": preds[splitted_idx["train"]]
    })
    val_score = evaluator.eval({
        "y_true": labels[splitted_idx["valid"]],
        "y_pred": preds[splitted_idx["valid"]]
    })
    test_score = evaluator.eval({
        "y_true": labels[splitted_idx["test"]],
        "y_pred": preds[splitted_idx["test"]]
    })

    return train_score['rocauc'], val_score['rocauc'], test_score['rocauc']

In [5]:
set_random_seed(0)

class Block(nn.Module):
    def __init__(self,in_feats,out_feats,num_channels=8):
        super(Block, self).__init__()
        self.gc = GraphConv(in_feats, out_feats)
        
    def forward(self, g, node_state):
        node_states = self.gc(g, node_state)
        node_states = F.relu(node_states)
        return node_states

class Net(nn.Module):
    def __init__(self, in_feats, n_hidden, out_feats,num_paths=2):
        super(Net, self).__init__()
        self.num_paths = num_paths
        self.init = Block(in_feats, n_hidden)
        self.lyrs = nn.ModuleList()
        for _ in range(6):
            self.lyrs.append(Block(n_hidden, n_hidden))
        self.lin = nn.Linear(n_hidden, out_feats)
        
    def forward(self, g):
        node_state = g.ndata['edata_mean']
        out = self.init(g, node_state)
        _out = out
        for i in range(6):
            out = self.lyrs[i](g, out)
        # for i in range(4,6):
        #     _out = self.lyrs[i](g, _out)
        # out = torch.stack([out, _out],dim=1).sum(1)
        out = self.lin(out)
        return out

random seed set to be 0


In [6]:
warmup_epochs = 10
num_epochs = 200
patience = 50
log_every = 10
lr = 1e-2
weight_decay = 0

model = Net(in_feats=8, n_hidden=256, out_feats=112).to(device)
graph = graph.to(device)

optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
min_lr = 1e-3
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=0.7, patience=200,  verbose=True, cooldown=20, min_lr=min_lr)
print ('scheduler min_lr', min_lr)
criterion = nn.BCEWithLogitsLoss()
evaluator = Evaluator('ogbn-proteins')


scheduler min_lr 0.001


In [7]:
dur = []
best_score = 0.
num_patient_epochs = 0
model_folder = './saved_models/'
model_path = model_folder + "model.pt"
log_path = "log.txt"

def printw(line):
    with open(log_path,'a') as f:
        f.write(line+"\n")
    print(line)

if not os.path.exists(model_folder):
    os.makedirs(model_folder)
    
print("Warming up for {:d} epochs...".format(warmup_epochs))
t0 = time.time()
for _ in range(warmup_epochs):
    loss, train_score = run_a_train_epoch(graph, splitted_idx["train"], model, criterion, optimizer, evaluator)
    scheduler.step(loss)
dur = time.time() - t0 
print("Done in {:.2f} sec".format(dur))

for epoch in range(1, num_epochs + 1):
    t0 = time.time()
    loss, train_score = run_a_train_epoch(graph, splitted_idx["train"], model, criterion, optimizer, evaluator)
    train_score, val_score, test_score = run_an_eval_epoch(graph, splitted_idx, model, evaluator)
    scheduler.step(val_score)
    dur = time.time() - t0   

    # Early stop
    if test_score > best_score:
        torch.save(model.state_dict(), model_path)
        best_score = test_score
        num_patient_epochs = 0
    else:
        num_patient_epochs += 1

    if (epoch%log_every) == 0:
        printw('Epoch {:d}\t loss {:.4f}\t train score {:.4f}\t val score {:.4f}\t test score {:.4f}\t dur {:.2f}s\t patience {:d}'.format(epoch, loss, train_score, val_score,test_score, dur, num_patient_epochs))

    if num_patient_epochs == patience:
        break

Warming up for 10 epochs...
Done in 58.38 sec
Epoch 10	 loss 0.4114	 train score 0.4661	 val score 0.3294	 test score 0.3710	 dur 12.28s	 patience 9
Epoch 20	 loss 0.3784	 train score 0.5440	 val score 0.4826	 test score 0.5407	 dur 11.95s	 patience 0
Epoch 30	 loss 0.3566	 train score 0.5976	 val score 0.6007	 test score 0.6339	 dur 12.29s	 patience 1
Epoch 40	 loss 0.3281	 train score 0.6431	 val score 0.6701	 test score 0.6757	 dur 12.34s	 patience 1
Epoch 50	 loss 0.3146	 train score 0.7284	 val score 0.7283	 test score 0.7014	 dur 12.46s	 patience 0
Epoch 60	 loss 0.3064	 train score 0.7499	 val score 0.7366	 test score 0.7045	 dur 12.58s	 patience 4
Epoch 70	 loss 0.3057	 train score 0.7544	 val score 0.7426	 test score 0.7084	 dur 13.39s	 patience 2
Epoch 80	 loss 0.3037	 train score 0.7570	 val score 0.7481	 test score 0.7099	 dur 12.75s	 patience 2
Epoch 90	 loss 0.3012	 train score 0.7614	 val score 0.7477	 test score 0.7112	 dur 12.67s	 patience 12
Epoch 100	 loss 0.2983	 tr

In [8]:
printw("Best test score: " +  str(best_score))

Best test score: 0.7388818116336939
